In [72]:
import pandas as pd

In [73]:
import catboost

In [74]:
catboost.__version__

'1.2'

In [32]:
df = pd.read_csv('../data/allocine_gold.csv')  #/home/apprenant/Bureau/ML_recap/data/allocine_gold.csv


In [33]:
import numpy as np

# Appliquer le logarithme naturel à la colonne 'actors_popularity'
df['actor_1_popularity'] = round(np.log1p(df['actor_1_popularity']), 6)
df['actor_2_popularity'] = round(np.log1p(df['actor_2_popularity']), 6)
df['actor_3_popularity'] = round(np.log1p(df['actor_3_popularity']), 6)
df['actors_popularity'] = round(np.log1p(df['actors_popularity']), 6)
df['director_popularity'] = round(np.log1p(df['director_popularity']), 6)
df['duree'] = round(np.log1p(df['duree']), 6)
# df['reputation_distributeur_1'] = round(np.log1p(df['reputation_distributeur_1']), 6)
# df['nombre_films_distributeur'] = round(np.log1p(df['nombre_films_distributeur']), 6)
df['moyenne_entrees_par_distributeur'] = round(np.log1p(df['moyenne_entrees_par_distributeur']), 6)

#df['budget'] = round(np.log1p(df['budget']), 6)
#df['budget'] = df['budget'] / df['budget'].max()


In [34]:
df.describe()

,note_presse,duree,nominations,prix,annee_production,Entrees_1ere_semaine,actor_1_popularity,actor_2_popularity,actor_3_popularity,director_popularity,actors_popularity,budget,moyenne_entrees_par_distributeur,reputation_distributeur,nombre_films_distributeur
count,7455.000000,7455.000000,7455.000000,7455.000000,7455.000000,7.455000e+03,7455.000000,7455.000000,7455.000000,7455.000000,7455.000000,7.455000e+03,7455.000000,7455.000000,7455.000000
mean,3.077855,4.631384,1.660496,1.850034,2012.682227,1.556107e+05,2.227870,2.116230,1.932261,1.266182,3.199356,1.545907e+07,10.982667,344.563380,132.301408
std,0.641621,0.223449,2.932011,4.651042,6.797704,3.001804e+05,1.088187,1.052864,1.018934,0.721432,1.057162,3.835177e+07,1.947137,90.136792,105.336689
min,1.000000,0.000000,0.000000,0.000000,1926.000000,4.000000e+00,0.470004,0.470004,0.470004,0.470004,1.029619,0.000000e+00,1.791759,1.000000,1.000000
25%,2.700000,4.521789,0.000000,0.000000,2009.000000,7.218500e+03,1.355320,1.280933,1.094939,0.683097,2.448243,0.000000e+00,9.863676,289.000000,36.000000
50%,3.100000,4.615121,0.000000,0.000000,2013.000000,4.923700e+04,2.199999,2.115050,1.830499,1.097278,3.186353,0.000000e+00,11.518059,381.000000,122.000000
75%,3.500000,4.727388,2.000000,0.000000,2018.000000,1.671530e+05,3.107184,2.962021,2.745699,1.699004,4.100642,1.000000e+07,12.501802,415.000000,181.000000
max,5.000000,6.298949,53.000000,62.000000,2023.000000,2.935984e+06,6.302194,6.302194,6.302194,4.476302,6.504564,4.600000e+08,14.236993,435.000000,405.000000


## Model Ml

In [35]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_hist_gradient_boosting  # Permet d'activer HistGradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor


In [36]:
df  = df.dropna()

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7455 entries, 0 to 7454
Data columns (total 30 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   titre                             7455 non-null   object 
 1   acteur_1                          7455 non-null   object 
 2   acteur_2                          7455 non-null   object 
 3   acteur_3                          7455 non-null   object 
 4   realisateur                       7455 non-null   object 
 5   distributeur                      7455 non-null   object 
 6   note_presse                       7455 non-null   float64
 7   duree                             7455 non-null   float64
 8   genre                             7455 non-null   object 
 9   pays                              7455 non-null   object 
 10  type                              7455 non-null   object 
 11  nominations                       7455 non-null   int64  
 12  prix  

In [38]:
df[["actor_1_popularity","actor_2_popularity","actor_3_popularity","director_popularity", 'type']]

,actor_1_popularity,actor_2_popularity,actor_3_popularity,director_popularity,type
0,3.525389,1.735365,0.830297,1.859574,Long-métrage
1,1.197552,0.870037,1.120374,0.603222,Long-métrage
2,1.934994,2.790613,0.839842,1.892359,Long-métrage
3,2.274597,3.627855,2.621475,1.521917,Long-métrage
4,1.392276,2.597119,0.936093,1.178039,Long-métrage
...,...,...,...,...,...
7450,2.315304,1.561298,2.022871,0.873801,Long-métrage
7451,1.356093,1.566113,1.111529,1.745192,Long-métrage
7452,2.097036,1.552444,2.086665,0.875469,Long-métrage
7453,0.505009,0.996949,1.119395,2.580520,Long-métrage


In [39]:
df.columns

Index(['titre', 'acteur_1', 'acteur_2', 'acteur_3', 'realisateur',
       'distributeur', 'note_presse', 'duree', 'genre', 'pays', 'type',
       'nominations', 'prix', 'annee_production', 'Semaine',
       'Entrees_1ere_semaine', 'actor_1_popularity', 'actor_2_popularity',
       'actor_3_popularity', 'director_popularity', 'vacances', 'saison',
       'actors_popularity', 'langue', 'budget', 'genre1', 'genre2',
       'moyenne_entrees_par_distributeur', 'reputation_distributeur',
       'nombre_films_distributeur'],
      dtype='object')

In [40]:
df.shape

(7455, 30)

In [41]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor

# # Séparer les colonnes catégorielles et numériques
# categorical_features = ['vacances', 'saison', 'pays', 'genre', 'annee_production', 'langue', 'type']
# numeric_features = ['actor_1_popularity', 'actor_2_popularity',
#        'actor_3_popularity','duree', 'director_popularity', 'reputation_distributeur', 'nombre_films_distributeur', 'budget']




categorical_features = ['vacances','saison','pays','annee_production','type','genre','genre1','langue']

numeric_features = ['actor_1_popularity', 'actor_2_popularity',
       'actor_3_popularity','duree', 'director_popularity',  'nombre_films_distributeur', 'budget','reputation_distributeur']





# Créer les transformers pour les colonnes catégorielles et numériques
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numeric_transformer = MinMaxScaler()

y = df['Entrees_1ere_semaine']
X = df[categorical_features+numeric_features]
# Créer le préprocesseur
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numeric_transformer, numeric_features)
    ]
)



pipline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', CatBoostRegressor())
])

# Paramètres pour CatBoostRegressor
param_distributions = {
    'regressor__iterations': [50, 100, 150, 200, 250],  # Nombre d'itérations d'entraînement
    'regressor__depth': [4, 6, 8, 10],  # Profondeur de l'arbre
    'regressor__learning_rate': [0.01, 0.05, 0.1, 0.2],  # Taux d'apprentissage
    'regressor__l2_leaf_reg': [1, 3, 5, 7]  # Régularisation L2
}


# Diviser les données en jeux d'entraînement (80%) et de test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Effectuer la recherche aléatoire des hyperparamètres
random_search = RandomizedSearchCV(pipline, param_distributions, n_iter=10, scoring='neg_mean_squared_error', cv=5, random_state=42)
random_search.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres trouvés
print("Meilleurs hyperparamètres:")
print(random_search.best_params_)

# Obtenir le meilleur modèle entraîné avec les meilleurs hyperparamètres
best_model = random_search.best_estimator_

# Faire des prédictions avec le meilleur modèle sur les données de test
y_pred = best_model.predict(X_test)


0:	learn: 286404.7148208	total: 5.54ms	remaining: 271ms
1:	learn: 267599.5056593	total: 11.1ms	remaining: 266ms
2:	learn: 253600.1666604	total: 16ms	remaining: 251ms
3:	learn: 240656.6837520	total: 20ms	remaining: 230ms
4:	learn: 231751.4554812	total: 25.1ms	remaining: 226ms
5:	learn: 224246.8080205	total: 29.2ms	remaining: 214ms
6:	learn: 218007.2218111	total: 32.7ms	remaining: 201ms
7:	learn: 211980.4138727	total: 37ms	remaining: 194ms
8:	learn: 208411.6915174	total: 41.5ms	remaining: 189ms
9:	learn: 203346.7404069	total: 45.2ms	remaining: 181ms
10:	learn: 200035.5895672	total: 48.8ms	remaining: 173ms
11:	learn: 195067.2922562	total: 52.4ms	remaining: 166ms
12:	learn: 191916.2563351	total: 56.3ms	remaining: 160ms
13:	learn: 189175.0676331	total: 61ms	remaining: 157ms
14:	learn: 187879.7235589	total: 64.7ms	remaining: 151ms
15:	learn: 186075.2851366	total: 69.6ms	remaining: 148ms
16:	learn: 184340.7781544	total: 74.4ms	remaining: 144ms
17:	learn: 181283.9020657	total: 78.3ms	remaining

In [42]:
from sklearn.metrics import r2_score
y_pred_train = best_model.predict(X_train)
print("######## accuracy_score : ")
print("TRAIN :", r2_score(y_train, y_pred_train))
print("TEST :", r2_score(y_test, y_pred))

######## accuracy_score : 
TRAIN : 0.8814081192404365
TEST : 0.623866897645379


In [43]:
df.shape

(7455, 30)

In [44]:
# import pickle

# # Enregistrez le meilleur modèle dans un fichier pickle
# with open('catboost.pkl', 'wb') as file:
#     pickle.dump(best_model, file)

In [45]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7455 entries, 0 to 7454
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   vacances                   7455 non-null   bool   
 1   saison                     7455 non-null   object 
 2   pays                       7455 non-null   object 
 3   annee_production           7455 non-null   int64  
 4   type                       7455 non-null   object 
 5   genre                      7455 non-null   object 
 6   genre1                     7455 non-null   object 
 7   langue                     7455 non-null   object 
 8   actor_1_popularity         7455 non-null   float64
 9   actor_2_popularity         7455 non-null   float64
 10  actor_3_popularity         7455 non-null   float64
 11  duree                      7455 non-null   float64
 12  director_popularity        7455 non-null   float64
 13  nombre_films_distributeur  7455 non-null   int64

In [47]:
X.head(1)

,vacances,saison,pays,annee_production,type,genre,genre1,langue,actor_1_popularity,actor_2_popularity,actor_3_popularity,duree,director_popularity,nombre_films_distributeur,budget,reputation_distributeur
0,False,Printemps,France,2023,Long-métrage,Thriller,Drama,fr,3.525389,1.735365,0.830297,4.663439,1.859574,125,0.0,362.0


In [51]:
int(best_model.predict(X.head(1))[0])

109226